In [ ]:
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, models, layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50V2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import io
from PIL import Image

In [ ]:
from minio import Minio
from minio.error import ResponseError

minioClient = Minio(
                'minio-service.kubeflow:9000',
                access_key='minio', 
                secret_key='minio123', 
                secure=False
            )

In [ ]:
INIT_LR = 1e-05  #1e-05  # This value is specific to what model is chosen: Inception, VGG or ResnNet.
EPOCHS = 30 
BS = 8

print("Loading images...")

data = []
labels = []

In [ ]:

# read all X-Rays in the specified path, and resize them all to 256x256

for i in minioClient.list_objects('dataset', prefix='covid19', recursive=True):
    label = i.object_name.split(os.path.sep)[-2]

    minioObj = minioClient.get_object('dataset', i.object_name)
    byteArray = minioObj.read()
    pil_image = Image.open(io.BytesIO(byteArray)).convert('RGB')
    image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))
    data.append(image)
    labels.append(label)

    
#normalise pixel values to real numbers between 0.0 - 1.0 
data = np.array(data) / 255.0
labels = np.array(labels)

# perform one-hot encoding for a 3-class labeling 
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
labels = to_categorical(integer_encoded)

print("... ... ", len(data), "images loaded in 3x classes:")
print(label_encoder.classes_)

In [ ]:
(x_train, x_val, y_train, y_val) = train_test_split(data, labels, test_size=0.20, stratify=labels)

In [ ]:
# Katib 파라미터
LEARNING_RATE = 0.00001 # List: 0.001, 0.0001, 0.0003, 0.00001, 0.00003
DENSE = 128             # Range: 50-200

In [ ]:
model = Sequential()
adam_s = Adam(learning_rate = LEARNING_RATE)

#model.add(VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='average'))
model.add(ResNet50V2(input_shape=(256, 256, 3),include_top=False, weights='imagenet',pooling='average'))

for layer in model.layers:
    layer.trainable = False

model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(DENSE, activation='relu'))
model.add(Dense(DENSE, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=adam_s, metrics=['accuracy'])


model.summary()

In [ ]:
# train the head of the network
print("Training the full stack model...")
H = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val), batch_size=8)

In [ ]:
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',   # Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0'    # If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

model.save("s3://model/new-covid/1")

# Serving Test

In [ ]:
print("[INFO] loading untrained test images...")
imagePathTest = "./all/test"
imagePathsTest = list(paths.list_images(imagePathTest))
print(len(imagePathsTest))

dataTest = []
labelsTest = []

In [ ]:
for i in minioClient.list_objects('dataset', prefix='covid19', recursive=True):
    labelTest = i.object_name.split(os.path.sep)[-2]

    minioObj = minioClient.get_object('dataset', i.object_name)
    byteArray = minioObj.read()
    pil_image = Image.open(io.BytesIO(byteArray)).convert('RGB')
    image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    imageTest = cv2.resize(image, (256, 256))
    dataTest.append(imageTest)
    labelsTest.append(labelTest)

In [ ]:
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 255]
dataTest = np.array(dataTest) / 255.0
labelsTest = np.array(labelsTest)


# perform one-hot encoding on the labels
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labelsTest)
labelsTest = to_categorical(integer_encoded)

print(labelsTest)

In [ ]:
predTest = model.predict(dataTest, batch_size=BS)
print(predTest)

In [ ]:
predClasses = predTest.argmax(axis=-1)
print(predClasses)

### Test용 payload json 생성

In [ ]:
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import cv2
import json
import requests
from tqdm import tqdm




#get some sample data
sample_test_data = dataTest[2]  #[0:2]
sample_test_labels = labelsTest[2] #[0:2]

print(sample_test_data.shape)

class_names = ['covid', 'normal', 'pneumonia_bac']

# pre-process data 
#sample_test_data_processed = np.expand_dims(sample_test_data / 255., axis=3)

sample_test_data_processed = sample_test_data

# create payload
data = json.dumps({"signature_name": "serving_default", 
                   "instances": sample_test_data_processed.tolist()})

In [ ]:
with open("image_data.json", "w") as json_file:
    json.dump(data, json_file)
# print(data)

### KFServing 테스트

In [ ]:
HEADERS = {'content-type': 'application/json'}
#MODEL1_API_URL = 'http://172.17.0.1:8511/v1/models/covid19/versions/2:predict'
MODEL1_API_URL = 'http://35.206.247.237:32380/v1/models/covid19:predict'

# inference request
json_response = requests.post(MODEL1_API_URL, data=data, headers=HEADERS)

print(json_response)

pred = json.loads(json_response.content.decode('utf-8'))
print(pred)

# view server response
predictions = json.loads(json_response.text)['predictions']
predictions = np.argmax(np.array(predictions), axis=1)
prediction_labels = [class_names[p] for p in predictions]

fig, ax = plt.subplots(6, 5, figsize=(60, 80))
for idx, img in enumerate(sample_test_data):
    rowidx = idx // 5
    colidx = idx % 5
    ax[rowidx, colidx].imshow(img)
    ax[rowidx, colidx].set_title('Actual: {}\nPredicted: {}'.format(class_names[sample_test_labels[idx].argmax(axis=-1)], 
                                                                    prediction_labels[idx]), fontsize=40)